In [ ]:
import mysql.connector
import random
from datetime import datetime, timedelta

# Połączenie z bazą danych
connection = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

cursor = connection.cursor()

# Zapytanie SQL do wstawiania danych do tabeli Rezerwacje
insert_data_query = """
INSERT INTO Rezerwacje (id_rezerwacje, id_klienci, id_pracownicy, id_wycieczki, id_transakcje, Data_rezerwacji)
VALUES (%s, %s, %s, %s, %s, %s);
"""

# Funkcja losująca datę rezerwacji w zakresie od 6 miesięcy do 1 tygodnia przed datą wyjazdu
def losuj_date_rezerwacji(data_rozpoczecia):
    # Określamy, że rezerwacja musi być co najmniej 1 tydzień przed datą rozpoczęcia wycieczki
    max_date = data_rozpoczecia - timedelta(weeks=1)
    # Określamy, że rezerwacja może być do 6 miesięcy przed datą rozpoczęcia wycieczki
    min_date = data_rozpoczecia - timedelta(weeks=26)
    
    # Losujemy datę rezerwacji pomiędzy min_date a max_date
    delta = max_date - min_date
    random_days = random.randint(0, delta.days)
    return min_date + timedelta(days=random_days)

# Pobranie wszystkich wycieczek z tabeli Realizowane_wycieczki
cursor.execute("SELECT id_wycieczki, Data_rozpoczecia FROM Realizowane_wycieczki")
wycieczki = cursor.fetchall()

# Pobranie wszystkich klientów z tabeli Klienci
cursor.execute("SELECT id_klienci FROM Klienci")
klienci = cursor.fetchall()

# Pobranie wszystkich pracowników z tabeli Pracownicy
cursor.execute("SELECT id_pracownicy FROM Pracownicy")
pracownicy = cursor.fetchall()

# Pobranie wszystkich transakcji z tabeli Transakcje_finansowe
cursor.execute("SELECT id_transakcje FROM Transakcje_finansowe")
transakcje = cursor.fetchall()

for i in range(1, 501):  # Generujemy 500 rezerwacji
    id_klienci = random.choice(klienci)[0]  # Losowanie klienta z tabeli Klienci
    id_pracownicy = random.choice(pracownicy)[0]  # Losowanie pracownika z tabeli Pracownicy
    id_wycieczki, data_rozpoczecia = random.choice(wycieczki)  # Losowanie wycieczki z tabeli Realizowane_wycieczki
    id_transakcje = random.choice(transakcje)[0]  # Losowanie transakcji z tabeli Transakcje_finansowe
    
    # Losowanie daty rezerwacji w zależności od daty rozpoczęcia wycieczki
    data_rezerwacji = losuj_date_rezerwacji(data_rozpoczecia).strftime('%Y-%m-%d')

    # Wstawianie danych do tabeli
    cursor.execute(insert_data_query, (i, id_klienci, id_pracownicy, id_wycieczki, id_transakcje, data_rezerwacji))

# Zatwierdzenie zmian w bazie danych
connection.commit()

# Zamknięcie kursora i połączenia
cursor.close()
connection.close()